In [ ]:
pip install plotly==4.14.3

In [40]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import date
from pymongo import MongoClient
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
client = MongoClient('mongo')
database = client['local']
collection = database['pixitag']

In [53]:
url = "http://www.pixiv.net/ranking.php?mode=daily"
response = requests.get(url)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36', 
          'password': 'rd.)T6vF',
          'pixiv_id' : 'flicfloclol@hotmail.fr'}
response = requests.get(url, headers=headers, timeout = 10)

parse = url.split('&')

imageId = []


if (len(parse) == 1):
    rankingDate = str(date.today())   
else:
    rankingDate = parse[1][5:9] + '-' + parse[1][9:11] + '-' + parse[1][11:13]


soup = BeautifulSoup(response.text)
links = []

# for single illustration (different class)
for link in soup.find_all('a', class_="work _work"):
    links.append("https://www.pixiv.net/ajax/illust/" + link.get('href').split('/')[2] + "?lang=en")
    imageId.append(link.get('href').split('/')[2])
    
# for multiple illustrations (different class)\n",
for link in soup.find_all('a', class_="work _work multiple"):
    links.append("https://www.pixiv.net/ajax/illust/" + link.get('href').split('/')[2] + "?lang=en")
    imageId.append(link.get('href').split('/')[2])



In [54]:
#url = "https://www.pixiv.net/ajax/ill?lang=enust/89101579"

data = []

counter = 0
imageCounter = 0

for url in links:

    response = requests.get(url, headers=headers, timeout = 100)
    jsonData = json.loads(response.text)

    tags = []

    for i in jsonData['body']['tags']['tags']:
        data.append({})
        if 'translation' not in i:
            if 'romaji' not in i:
                data[counter].update({'tag': i['tag']})
            else:
                data[counter].update({'tag': i['romaji']})
        else:
            data[counter].update({'tag': i['translation']['en']})
            
        data[counter].update({'date' : rankingDate})
        data[counter].update({'username' : jsonData['body']['userName']})
        data[counter].update({'imageId' : imageId[imageCounter]})
        counter += 1
    imageCounter += 1
    

print(data)


[{'tag': 'Gold Ship', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'gorushi', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'Uma Musume Pretty Derby', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'horse girl', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'high boots', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'korekosogago-rudoshippu', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'armpits', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'umamusumepurithi-da-bi-ichimannyu-za-iri', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'Gold Ship (UMPD)', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': '寒いとドラミングかます美少女', 'date': '2021-04-18', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'scenery', 'date': '20

In [55]:
cursor = collection.find({"date": rankingDate}, {"imageId", 'tag'})
ids = []
for document in cursor:
    ids.append(document)

for i in data:
    if i:
        for dbData in ids:
            if dbData['imageId'] == i['imageId'] and dbData['tag'] == i['tag']:
                i = {}
                break
        if i:
            collection.insert_one(i)
            print('Insert: ' + i['imageId'])


Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89175394
Insert: 89175394
Insert: 89175394
Insert: 89175393
Insert: 89175393
Insert: 89175393
Insert: 89175393
Insert: 89175393
Insert: 89175393
Insert: 89175393
Insert: 89175393
Insert: 89174996
Insert: 89179417
Insert: 89179417
Insert: 89175768
Insert: 89175768
Insert: 89212140
Insert: 89212140
Insert: 89212140
Insert: 89212140
Insert: 89212140
Insert: 89212140
Insert: 89212140
Insert: 89212140
Insert: 89199251
Insert: 89174589
Insert: 89174589
Insert: 89174589
Insert: 89174589
Insert: 89174589
Insert: 891885

In [56]:
cursor = collection.find({'tag': 'original 3000+ bookmarks'})
ids = []
for document in cursor:
    ids.append(document)
    print(document)
    print('--------')

{'_id': ObjectId('607b0aae5c82710b90465f2f'), 'tag': 'original 3000+ bookmarks', 'date': '2021-04-01', 'username': 'ポ～ン（出水ぽすか）', 'imageId': '88847570'}
--------
{'_id': ObjectId('607b0aae5c82710b90465fab'), 'tag': 'original 3000+ bookmarks', 'date': '2021-04-01', 'username': '碧風羽 BOOTH店オープン', 'imageId': '88841115'}
--------


In [18]:
collection.remove()
print(collection)

Collection(Database(MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True), 'local'), 'pixitag')


<ipython-input-18-ea60bee885b1>:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  collection.remove()


In [85]:
#Get most popular tag by date
cursor = collection.aggregate([
    {"$match": {'date': '2021-04-17'}},
    {'$group': {
        '_id': '$tag',
        'nb': {'$sum': 1}
    }},
    {'$sort': {'nb': -1}}
])
for document in cursor:
    ids.append(document)
    print(document)

{'_id': 'original', 'nb': 14}
{'_id': 'girl', 'nb': 6}
{'_id': 'Uma Musume Pretty Derby', 'nb': 5}
{'_id': 'Fate/GrandOrder', 'nb': 4}
{'_id': 'FGO', 'nb': 4}
{'_id': 'original 5000+ bookmarks', 'nb': 3}
{'_id': 'Virtual Youtuber', 'nb': 3}
{'_id': 'clothed breasts', 'nb': 3}
{'_id': 'Gold Ship (UMPD)', 'nb': 3}
{'_id': 'evangelion', 'nb': 3}
{'_id': 'Fate/GO 1000+ bookmarks', 'nb': 3}
{'_id': 'umaamusumepurithi-da-bi-sennyu-za-iri', 'nb': 2}
{'_id': 'Genshin Impact', 'nb': 2}
{'_id': 'subarashikihokkorinosekai', 'nb': 2}
{'_id': 'Evangelion New Theatrical Release', 'nb': 2}
{'_id': 'Evangelion 1000+ bookmarks', 'nb': 2}
{'_id': 'evagosennyu-za-zuiri', 'nb': 2}
{'_id': 'Shikinami Asuka Langley', 'nb': 2}
{'_id': 'Pokémon', 'nb': 2}
{'_id': 'Ashiya Douman (Fate)', 'nb': 2}
{'_id': 'Nijisanji', 'nb': 2}
{'_id': 'creation', 'nb': 2}
{'_id': 'korekosogago-rudoshippu', 'nb': 2}
{'_id': 'Project Sekai', 'nb': 2}
{'_id': 'work illustration', 'nb': 2}
{'_id': 'plugsuit', 'nb': 2}
{'_id': 'Lumi

In [47]:
#Get images number per illustrator in ranking
cursor = collection.aggregate([
    #{"$match": {'date': '2021-04-17'}},
    {'$group': {
        '_id': {'username': '$username'},
        'images': {'$addToSet': '$imageId'}
    }},
    {'$project': {'nb': {'$size': '$images'}}},
    {'$sort': {'nb': -1}}
])
graphData = {'username': [], 'nb': []}
for document in cursor:
    #graphData.update({document['_id']['username']: [document['nb']]})
    graphData['username'].append(document['_id']['username'])
    graphData['nb'].append(document['nb'])
    #print(document)
#print(graphData)

In [48]:
#x = ['Product A', 'Product B', 'Product C']
#y = [20, 14, 23]

# Use the hovertext kw argument for hover text
fig = go.Figure(data=[go.Bar(x=graphData['username'], y=graphData['nb'])])
# Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Image number per illustrator in ranking')
fig.show()

In [69]:
#Get 
cursor = collection.aggregate([
    {'$group': {
        '_id': {'tag': '$tag'},
        'date': {'$addToSet': '$date'}
    }},
    {'$project': {
        'nb': {'$size': '$date'}
    }},
    {'$match': {
        'nb': {'$gt': 3}
    }},
    {'$sort': {'nb': -1}}
])
graphData = {'tag': [], 'nb': []}
for document in cursor:
    graphData['tag'].append(document['_id']['tag'])
    graphData['nb'].append(document['nb'])
    print(document)

{'_id': {'tag': 'horse girl'}, 'nb': 5}
{'_id': {'tag': 'Pokémon'}, 'nb': 5}
{'_id': {'tag': 'work illustration'}, 'nb': 5}
{'_id': {'tag': 'original'}, 'nb': 5}
{'_id': {'tag': 'girl'}, 'nb': 5}
{'_id': {'tag': 'evangelion'}, 'nb': 5}
{'_id': {'tag': 'Uma Musume Pretty Derby'}, 'nb': 5}
{'_id': {'tag': 'subarashikihokkorinosekai'}, 'nb': 5}
{'_id': {'tag': 'beautiful'}, 'nb': 5}
{'_id': {'tag': 'breasts'}, 'nb': 5}
{'_id': {'tag': 'original 5000+ bookmarks'}, 'nb': 5}
{'_id': {'tag': 'copyright'}, 'nb': 5}
{'_id': {'tag': 'gloves'}, 'nb': 5}
{'_id': {'tag': 'scenery'}, 'nb': 5}
{'_id': {'tag': 'Genshin Impact'}, 'nb': 5}
{'_id': {'tag': 'neon genesis evangelion'}, 'nb': 5}
{'_id': {'tag': 'ここに巣を作ろう'}, 'nb': 5}
{'_id': {'tag': 'creation'}, 'nb': 5}
{'_id': {'tag': 'korehaiihatsune'}, 'nb': 4}
{'_id': {'tag': 'manga'}, 'nb': 4}
{'_id': {'tag': 'young girl'}, 'nb': 4}
{'_id': {'tag': 'umaamusumepurithi-da-bi-sennyu-za-iri'}, 'nb': 4}
{'_id': {'tag': 'plugsuit'}, 'nb': 4}
{'_id': {'tag': 

In [70]:
fig = go.Figure(data=[go.Bar(x=graphData['tag'], y=graphData['nb'])])
# Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(0,0,0)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Tag popularity by number of days')
fig.show()